In [1]:
import import_ipynb
from import_data import get_all_binance
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

importing Jupyter notebook from import_data.ipynb


In [2]:
def init():    
    global tokens
    tokens = {}
    tokens['Bitcoin'] = {'symbol':'BTC',
                         'Reference':'https://bitcoin.org/en/'
                        }
    tokens['Ethereum'] = {'symbol':'ETH',
                         'Reference':'https://ethereum.org'
                        }
    tokens['Litecoin'] = {'symbol':'LTC',
                         'Reference':'https://litecoin.org'
                        }
    tokens['Cardano'] = {'symbol':'ADA',
                         'Reference':'https://www.cardano.org/en/home/'
                        }
    tokens['Ripple'] = {'symbol':'XRP',
                         'Reference':'https://ripple.com'
                        }
    tokens['EOSIO'] = {'symbol':'EOS',
                         'Reference':'https://eos.io'
                        }
    tokens['IOTA'] = {'symbol':'IOTA',
                         'Reference':'https://www.iota.org'
                        }
    tokens['Stellar'] = {'symbol':'XLM',
                         'Reference':'https://www.stellar.org'
                        }
    tokens['Tron'] = {'symbol':'TRX',
                         'Reference':'https://tron.network'
                        }
    tokens['Chainlink'] = {'symbol':'LINK',
                         'Reference':'https://chain.link'
                        }
    tokens['Monero'] = {'symbol':'XMR',
                         'Reference':'https://www.getmonero.org'
                        }
    tokens['Dash'] = {'symbol':'DASH',
                         'Reference':'https://www.dash.org'
                        }
    tokens['Tezos'] = {'symbol':'XTZ',
                         'Reference':'https://www.tezos.com'
                        }

    global tickers
    tickers = []
    for token in tokens:
        tickers.append(tokens[token]['symbol']+'USDT')

In [3]:
def update_data():
    for symbol in tickers:
        get_all_binance(symbol, '1d', save = True)

In [4]:
def show_tokens():
    print('The following are the available tokens to build your portfolio:\n')
    print("%10s %10s %40s" % ('Name','Symbol', 'Reference'))
    print('-'*62)
    for token in tokens:
        #print(token +'\t\t'+ tokens[token]['symbol'] +'\t\t'+ tokens[token]['Reference'])
        print("%10s %10s %40s" % (token,tokens[token]['symbol'],tokens[token]['Reference']))

In [5]:
def open_prices():
    complete_data = {}
    for ticker in tickers:
        complete_data[ticker] = pd.read_csv(ticker+'-1d-data.csv')[['timestamp', 'open']]
        complete_data[ticker].rename(columns = {'open':ticker}, inplace=True)
    
    open_price_data = complete_data[tickers[0]][['timestamp']]
    for ticker in tickers:
        open_price_data = pd.merge(open_price_data, complete_data[ticker], how='inner', on='timestamp')    
    
    return open_price_data

In [6]:
def portfolio_value(portfolio):
    prices = open_prices()
    portfolio_value = prices['BTCUSDT']*portfolio['BTC']
    for token in tokens:
        symbol = tokens[token]['symbol']
        if symbol != 'BTC':
            symbol_usd = symbol + 'USDT'
            portfolio_value = portfolio_value + prices[symbol_usd]*portfolio[symbol]
    return pd.DataFrame(data={'Value':portfolio_value.values}, index=prices['timestamp'].values)

In [7]:
# https://towardsdatascience.com/learn-to-calculate-your-portfolios-value-at-risk-e1e2c5c68456

def CalculateVaR_ES(portfolio_value, Percentile=99, Days=1):
    current_value = portfolio_value['Value'][-1]
    # calculate all possible values with historical rate of returns
    dollar_change = portfolio_value['Value'].pct_change().values[1:]*portfolio_value['Value'][-1]
    dollar_change.sort()
    ValueLocForPercentile = round(len(dollar_change) * (1 - (Percentile / 100)))
    
    # we assume normally iid observations, so we can multiply by sqrt(Days) to extend the loss to a given number of days
    VaR = dollar_change[ValueLocForPercentile + 1]*np.sqrt(Days)
    ES = dollar_change[0:ValueLocForPercentile].mean()*np.sqrt(Days)
    
    return VaR, ES